<a href="https://colab.research.google.com/github/Soichiro-Gardinner/Yelp_API/blob/main/Efficient_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, math, time
import json as js
from tqdm.notebook import tqdm_notebook

In [ ]:
!pip install yelpapi

In [ ]:
from yelpapi import YelpAPI as yalp

In [ ]:
# Load API Creadentials using Json:
with open("/content/YELP_Api_Key.json") as f:
  login = js.load(f)

login.keys()

dict_keys(['Key'])

In [ ]:
# Instantiate YelpAPI Variable:
yelp_api = yalp(login['Key'], timeout_s=5.0)

In [ ]:
# set our API call parameters
LOCATION = 'NY,NY'
TERM = 'Burger'

In [ ]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "/content/results_in_progress_NY_Burgers.json"
JSON_FILE

'/content/results_in_progress_NY_Burgers.json'

In [ ]:
## Check if Json file exists:
file_exists = os.path.isfile(JSON_FILE) #The os module checks if the file exists
## If it does not exist:
if file_exists == False:
  ## Create Any Needed Folders:
  # Get the Foldr Name Only
  Folder = os.path.dirname(JSON_FILE)
  ## If json file included a folder:
  if len(Folder)>0:
    # Create the folder:
    os.makedirs(Folder,exist_ok=True)
  print(f"[i] {JSON_FILE} not found. Saving empty list to file.")

  # Save an Empty list:
  with open(JSON_FILE,"w") as f:
    js.dump([],f)
  # If it Exists, inform user
else:
  print(f"[i]{JSON_FILE} already exists.")

[i] /content/results_in_progress_NY_Burgers.json not found. Saving empty list to file.


In [ ]:
## Load previous results and use len of results for offset
with open(JSON_FILE, "r") as f:
  previous_results = js.load(f)

## set offset based on previous result:
n_results = len(previous_results)
print(f"- {n_results} previous results found.")

- 0 previous results found.


In [ ]:
results = yelp_api.search_query(location = LOCATION
                            ,term = TERM,
                            offset= n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [ ]:
## How many results total?
total_results = results['total']
total_results

16900

In [ ]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [ ]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

845

In [ ]:
# Add the new results to the old list by using .extend()
previous_results.extend(results["businesses"])
with open (JSON_FILE, "w") as f:
  js.dump(previous_results, f)

**For Loop to call and concatenate each page:**

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):

    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = js.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)

    ## append new results and save to file
    previous_results.extend(results['businesses'])

    with open(JSON_FILE,'w') as f:
        js.dump(previous_results,f)

    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/845 [00:00<?, ?it/s]

YelpAPIError: ignored

In [ ]:
a = n_results + results_per_page
a

1020

**Function From the dojo to delete the JSON_FILE if it exists.**

In [ ]:
# This efficient Fubction takes a file as the 1st parameter to
## Check its existence on the folde, and a boolean variable to determine if
### we want to delete our files or not.

def create_json_file(FILE,  delete_if_exists=False):

    ## Check if FILE exists
    file_exists = os.path.isfile(FILE)

    ## If it DOES exist:
    if file_exists == True:

        ## Check if user wants to delete if exists
        if delete_if_exists==True:

            print(f"[!] {FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(FILE)
            # Recursive call to function after old file deleted
            create_json_file(FILE,delete_if_exists=False)
        else:
            print(f"[i] {FILE} already exists.")


    ## If it does NOT exist:
    else:

        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {FILE} not found. Saving empty list to new file.")

        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(FILE)

        ## If FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(FILE,'w') as f:
            js.dump([],f)

In [ ]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = js.load(f)

## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
print(f"\nTotal Number of Pages: {n_pages}")
print(previous_results)



[!] /content/results_in_progress_NY_Burgers.json already exists. Deleting previous file...
[i] /content/results_in_progress_NY_Burgers.json not found. Saving empty list to new file.
- 0 previous results found.

Total Number of Pages: 875
[]


- Note How this Time we got an 885 this time instead of 885 from our 'n_pages' varianble.

### **Loop to check if we exceed the limit and stop before we get an error.**

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):

    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = js.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)

    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    ## use n_results as the OFFSET
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM,
                                    offset=n_results)



    ## append new results and save to file
    previous_results.extend(results['businesses'])

    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        js.dump(previous_results,f)

    time.sleep(.2)

  0%|          | 0/875 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [ ]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,VVYea3NzbklOyHEzSEavWw,7th-street-burger-new-york-9,7th Street Burger,https://s3-media1.fl.yelpcdn.com/bphoto/un0Pg-...,False,https://www.yelp.com/biz/7th-street-burger-new...,87,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 40.75238, 'longitude': -73.98939}","[delivery, pickup]","{'address1': '485 7th Ave', 'address2': None, ...",,,5113.650606,NaN
1,8xpGj_YkI2X6QxqXUJPw7g,7th-street-burger-new-york,7th Street Burger,https://s3-media2.fl.yelpcdn.com/bphoto/GYqi52...,False,https://www.yelp.com/biz/7th-street-burger-new...,287,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.72697, 'longitude': -73.98546}","[delivery, pickup]","{'address1': '91 E 7th St', 'address2': None, ...",+16464906797,(646) 490-6797,2280.348538,$$
2,H26qMBb0OFNn2xmSTM1Phw,kings-of-kobe-new-york-new-york-2,Kings of Kobe - New York,https://s3-media1.fl.yelpcdn.com/bphoto/E09YEC...,False,https://www.yelp.com/biz/kings-of-kobe-new-yor...,432,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 40.761172, 'longitude': -74.000307}","[delivery, pickup]","{'address1': '650 W 42nd St', 'address2': None...",+16463705121,(646) 370-5121,6283.378163,$$
3,9whm9oIO5baMzRtpJhzNZA,the-york-new-york,The York,https://s3-media3.fl.yelpcdn.com/bphoto/9GKSWA...,False,https://www.yelp.com/biz/the-york-new-york?adj...,8,"[{'alias': 'gastropubs', 'title': 'Gastropubs'}]",5.0,"{'latitude': 40.727878, 'longitude': -73.979413}",[],"{'address1': '186 Avenue B', 'address2': '', '...",,,2322.545322,NaN
4,d_EKp-q91CQQFL86I_DuBw,burgers-on-b-new-york,Burgers on B,https://s3-media2.fl.yelpcdn.com/bphoto/qKa4um...,False,https://www.yelp.com/biz/burgers-on-b-new-york...,10,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 40.727244999999996, 'longitude': ...","[delivery, pickup]","{'address1': '168 Avenue B', 'address2': '', '...",,,2256.950452,NaN


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
995,n9uh27lhZp41ARuHCaKGHQ,dips-brooklyn,Dips,https://s3-media1.fl.yelpcdn.com/bphoto/gzjl49...,False,https://www.yelp.com/biz/dips-brooklyn?adjust_...,9,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",4.0,"{'latitude': 40.6322899, 'longitude': -73.9945...",[delivery],"{'address1': '5315 13th Ave', 'address2': '', ...",+17184352411,(718) 435-2411,8404.009543,$$
996,0AoaUlXkQOlalpA3v-Ivqw,lava-shawarma-new-york,Lava Shawarma,https://s3-media3.fl.yelpcdn.com/bphoto/CzE3dW...,False,https://www.yelp.com/biz/lava-shawarma-new-yor...,226,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 40.71964, 'longitude': -73.98507}","[delivery, pickup]","{'address1': '57 Clinton St', 'address2': '', ...",+16465594250,(646) 559-4250,1493.250393,$
997,RbMC9zD11aB3kgrXi6O3BQ,the-narrows-brooklyn,The Narrows,https://s3-media3.fl.yelpcdn.com/bphoto/gYIlra...,False,https://www.yelp.com/biz/the-narrows-brooklyn?...,188,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.704114, 'longitude': -73.930878}",[delivery],"{'address1': '1037 Flushing Ave', 'address2': ...",+12818271800,(281) 827-1800,4084.570644,$$
998,OefJvbJxLW1ZTr-8HUZdyQ,the-grotto-brooklyn,The Grotto,https://s3-media2.fl.yelpcdn.com/bphoto/2Sp25o...,False,https://www.yelp.com/biz/the-grotto-brooklyn?a...,29,"[{'alias': 'caribbean', 'title': 'Caribbean'},...",3.5,"{'latitude': 40.68734, 'longitude': -73.95083}","[delivery, pickup]","{'address1': '321 Nostrand Ave', 'address2': '...",+13472953446,(347) 295-3446,3237.823588,NaN
999,owvn3TwI2oweLWh4D7strA,comfortland-astoria-2,Comfortland,https://s3-media2.fl.yelpcdn.com/bphoto/bkJWAQ...,False,https://www.yelp.com/biz/comfortland-astoria-2...,326,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",4.0,"{'latitude': 40.763865, 'longitude': -73.914546}","[delivery, pickup]","{'address1': '40-09 30th Ave', 'address2': '',...",+13476429932,(347) 642-9932,8347.206960,$$


In [ ]:
# check for duplicate ID's
final_df.duplicated(subset='id').sum()

237

In [ ]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [ ]:
# save the final results to a compressed csv
final_df.to_csv('/content/results_in_progress_NY_Burgers.json', compression='gzip',index=False)